In [80]:
import faiss
import numpy as np
import pandas as pd
import scipy
from scipy.spatial.distance import cosine
import random
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
%matplotlib inline

In [81]:
DATASET = 'data/LOCALHIST_CORRUPT'
TRAIN_CSV = DATASET+'_df.csv'
SUBCLIPS_CSV = DATASET+'_subclips.csv'

In [82]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

,video_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_890,x_891,x_892,x_893,x_894,x_895,x_896,x_897,x_898,x_899
0,../data/1943 - Victory Through Air Power.avi,0.033367,0.000000,0.0,0.0,0.000000,0.000000,0.140162,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../data/1943 - Victory Through Air Power.avi,2.035369,0.000108,0.0,0.0,0.000000,0.000044,0.007993,0.000000,0.000355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,../data/1943 - Victory Through Air Power.avi,2.068735,0.000058,0.0,0.0,0.000105,0.000036,0.009339,0.000008,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,../data/1943 - Victory Through Air Power.avi,2.102102,0.000017,0.0,0.0,0.000147,0.000103,0.008468,0.000019,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../data/1943 - Victory Through Air Power.avi,2.135469,0.000006,0.0,0.0,0.000203,0.000050,0.008313,0.000011,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
subclips_df = pd.read_csv(SUBCLIPS_CSV)
subclips_df.head()

,clip_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_891,x_892,x_893,x_894,x_895,x_896,x_897,x_898,x_899,video_path
0,./tmp/1943 - Victory Through Air Power.avi_sub...,0.000000,0.000003,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../data/1943 - Victory Through Air Power.avi
1,./tmp/1943 - Victory Through Air Power.avi_sub...,7.173841,0.000380,0.001302,0.000158,0.000158,0.0,0.0,0.000278,0.001946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../data/1943 - Victory Through Air Power.avi
2,./tmp/1943 - Victory Through Air Power.avi_sub...,7.207207,0.000003,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../data/1943 - Victory Through Air Power.avi
3,./tmp/1943 - Victory Through Air Power.avi_sub...,7.807808,0.000003,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../data/1943 - Victory Through Air Power.avi
4,./tmp/1943 - Victory Through Air Power.avi_sub...,7.874541,0.000003,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,../data/1943 - Victory Through Air Power.avi


In [84]:
subclips_df.video_path.value_counts()

../data/1948 - Melody Time.avi                               244
../data/1943 - Victory Through Air Power.avi                 184
../data/1945 - The Three Caballeros.avi                      135
../data/1940 - Pinocchio.avi                                  50
../data/1948 - So Dear to My Heart.mp4                        46
../data/1947 - Fun and Fancy Free.avi                         43
../data/1937 - Snow White and the Seven Dwarves.avi           41
../data/1949 - The Adventures Of Ichabod And Mr. Toad.m4v     37
../data/The.Young.Pope.S01E02.HDTVRip.Jaskier.avi             24
../data/1941 - Dumbo.avi                                      22
../data/The.Young.Pope.S01E01.HDTVRip.Jaskier.avi             22
../data/1942 - Bambi.avi                                      19
../data/The.Young.Pope.S01E04.HDTVRip.Jaskier.avi             19
../data/1946 - Make Mine Music.avi                            18
../data/The.Young.Pope.S01E03.HDTVRip.Jaskier.avi             18
../data/1928 - Mickey Mou

In [85]:
def get_closest_vectors(query_vectors, vectors, metric):
    distances = scipy.spatial.distance.cdist(query_vectors, vectors, metric=metric)
    min_indices = np.argmin(distances, axis=1)
    min_dists = distances[range(len(distances)), min_indices]
    return min_indices, min_dists

class VideoClassifier:
    def __init__(self, vectors, labels, treshold=0.3, metric='euclidean'):
        """
        N - total amount of video frames
        K - amount of features
        vectors: N x K frame descriptor vectors
        labels: N x 1 frame labels
        """
        self.vectors = vectors
        self.labels = labels
        self.treshold = treshold
        self.metric = metric
        
    def classify(self, vectors):
        """
        Majority vote
        """
        min_indices, min_dists = get_closest_vectors(vectors, self.vectors, self.metric)
        votes = self.labels[min_indices]
        miss_mask = min_dists > self.treshold
        votes[miss_mask] = 'miss'
        moc = max([(list(votes).count(chr),chr) for chr in set(votes)])
        moc = moc[1]
        return moc, votes, min_dists

In [86]:
class FaissVideoClassifier:
    def __init__(self, vectors, labels, treshold=0.3, ncells=0.01):
        """
        N - total amount of video frames
        K - amount of features
        vectors: N x K frame descriptor vectors
        labels: N x 1 frame labels
        """
        vectors = np.ascontiguousarray(vectors.astype('float32'))
        faiss.normalize_L2(vectors)
        self.vectors = np.ascontiguousarray(vectors)
        
        quantizer = faiss.IndexFlatL2(self.vectors.shape[1])  # the other index
        self.index = faiss.IndexIVFFlat(quantizer, self.vectors.shape[1], int(ncells*len(self.vectors)), faiss.METRIC_INNER_PRODUCT)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        self.labels = labels
        self.treshold = treshold
        
    def classify(self, vectors):
        """
        Majority vote
        """
        vectors = np.ascontiguousarray(vectors.astype('float32'))
        faiss.normalize_L2(vectors)
        vectors = np.ascontiguousarray(vectors)
        D, I = self.index.search(vectors, 1)
        min_indices, min_dists = I.flatten(), D.flatten()
        closest_vectors = self.vectors[min_indices]

        votes = self.labels[min_indices]
        miss_mask = abs(1 - min_dists) > self.treshold
        votes[miss_mask] = 'miss'
        
        moc = max([(list(votes).count(chr),chr) for chr in set(votes)])
        moc = moc[1]
        return moc, votes, min_dists

In [87]:
train_df.head()

,video_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_890,x_891,x_892,x_893,x_894,x_895,x_896,x_897,x_898,x_899
0,../data/1943 - Victory Through Air Power.avi,0.033367,0.000000,0.0,0.0,0.000000,0.000000,0.140162,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../data/1943 - Victory Through Air Power.avi,2.035369,0.000108,0.0,0.0,0.000000,0.000044,0.007993,0.000000,0.000355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,../data/1943 - Victory Through Air Power.avi,2.068735,0.000058,0.0,0.0,0.000105,0.000036,0.009339,0.000008,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,../data/1943 - Victory Through Air Power.avi,2.102102,0.000017,0.0,0.0,0.000147,0.000103,0.008468,0.000019,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../data/1943 - Victory Through Air Power.avi,2.135469,0.000006,0.0,0.0,0.000203,0.000050,0.008313,0.000011,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
frame_times = train_df['frame_time']
labels = train_df.video_path.values
frame_vectors = np.ascontiguousarray(train_df.drop(['frame_time', 'video_path'], axis=1).values).astype('float32')
frame_vectors

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0824373e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.8285088e-05, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [5.7452440e-04, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 8.3264413e-06, 8.3264413e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.9428362e-04, 0.0000000e+00, 5.8285088e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [89]:
clf = FaissVideoClassifier(frame_vectors, labels, treshold=0.5)

In [90]:
%%time
unique_subclips = list(subclips_df.clip_path.unique())
y_true = []
y_pred = []
for clip in unique_subclips:
    subclip_df = subclips_df[subclips_df.clip_path == clip]
    subclip_vectors =  subclip_df.drop(['frame_time', 'video_path', 'clip_path'], axis=1).values
    true_label = subclip_df.iloc[0].video_path
    y_true.append(true_label)
    predicted_label, votes, dists = clf.classify(subclip_vectors)
    y_pred.append(predicted_label)

CPU times: user 3.17 s, sys: 92.9 ms, total: 3.26 s
Wall time: 947 ms


In [91]:
print(classification_report(y_true, y_pred))

                                                           precision    recall  f1-score   support

      ../data/1928 - Mickey Mouse -  Steamboat Willie.mp4       1.00      0.20      0.33        10
            ../data/1929 - Mickey Mouse - Plane Crazy.avi       1.00      1.00      1.00        10
      ../data/1937 - Snow White and the Seven Dwarves.avi       1.00      0.80      0.89        10
                              ../data/1940 - Fantasia.avi       1.00      0.67      0.80         3
                             ../data/1940 - Pinocchio.avi       0.71      1.00      0.83        10
                                 ../data/1941 - Dumbo.avi       0.82      0.90      0.86        10
                  ../data/1941 - The Reluctant Dragon.avi       0.67      0.50      0.57         8
                                 ../data/1942 - Bambi.avi       0.73      0.89      0.80         9
             ../data/1943 - Victory Through Air Power.avi       1.00      0.90      0.95        10
         

In [92]:
print('Accuracy', accuracy_score(y_true, y_pred))

[[ 2  0  0  0  1  0  2  0  0  0  0  0  1  0  2  0  1  0  1  0]
 [ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  4  1  0  0  0  0  0  0  1  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  9  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  2  0  0  6  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  8  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  1  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0

In [65]:
# Removing one video from the training set, so that we can see how the algorithm behaves on out-of-sample data

In [66]:
to_remove = random.sample(list(train_df.video_path.unique()), 3)
cut_df = train_df[~train_df.video_path.isin(to_remove)]

frame_times = cut_df.frame_time
labels = cut_df.video_path.values
frame_vectors = cut_df.drop(['frame_time', 'video_path'], axis=1).values

In [97]:
clf = FaissVideoClassifier(frame_vectors, labels, treshold=0.3)

In [98]:
targets = cut_df.video_path.unique()
unique_subclips = list(subclips_df.clip_path.unique())
y_true = []
y_pred = []
for clip in unique_subclips:
    subclip_df = subclips_df[subclips_df.clip_path == clip]
    subclip_vectors =  subclip_df.drop(['frame_time', 'video_path', 'clip_path'], axis=1).values
    true_label = subclip_df.iloc[0].video_path if subclip_df.iloc[0].video_path in targets else 'miss'
    y_true.append(true_label)
    predicted_label, votes, dists = clf.classify(subclip_vectors)
    y_pred.append(predicted_label)
print(classification_report(y_true, y_pred))
print('Accuracy', accuracy_score(y_true, y_pred))

                                                           precision    recall  f1-score   support

      ../data/1928 - Mickey Mouse -  Steamboat Willie.mp4       1.00      0.20      0.33        10
            ../data/1929 - Mickey Mouse - Plane Crazy.avi       1.00      1.00      1.00        10
      ../data/1937 - Snow White and the Seven Dwarves.avi       1.00      0.70      0.82        10
                              ../data/1940 - Fantasia.avi       1.00      0.67      0.80         3
                             ../data/1940 - Pinocchio.avi       0.00      0.00      0.00         0
                                 ../data/1941 - Dumbo.avi       1.00      0.90      0.95        10
                  ../data/1941 - The Reluctant Dragon.avi       0.00      0.00      0.00         0
                                 ../data/1942 - Bambi.avi       0.89      0.89      0.89         9
             ../data/1943 - Victory Through Air Power.avi       0.00      0.00      0.00         0
         